In [1]:
ACCESSORIES_TO_COLUMN_NAME = {
    'Airconditioning: Automatic': 'airconditioning',
    'Parking sensors': 'parking_sensors',
    'Satellite navigator': 'satellite_navigator',
    'Cruise control: Adaptive': 'cruise_control_adaptive',
    'Parking camera: Simple camera': 'parking_camera_simple_camera',
    'Seat heaters': 'seat_heaters',
    'Driving assistant': 'driving_assistant',
    'Alloy wheels': 'alloy_wheels',
    'Electric mirrors': 'electric_mirrors',
    'Lane departure warning system': 'lane_departure_warning_system',
    'Electrically operated tailgate': 'electrically_operated_tailgate',
    'Leather upholstery': 'leather_upholstery',
    'Emergency brake assist': 'emergency_brake_assist',
    'Collision avoidance system': 'collision_avoidance_system',
    'Cruise control: Traditional': 'cruise_control_traditional',
    'Parking Assistant': 'parking_assistant',
    'Tow bar': 'tow_bar',
    'Electric seats: With memory': 'electric_seats_with_memory',
    'Heated steering wheel': 'heated_steering_wheel',
    'Sunroof': 'sunroof',
    'Sun hatch: With panorama': 'sun_hatch_with_panorama',
    'Adaptive headlights': 'adaptive_headlights',
    'Sport seats': 'sport_seats',
    'Fuel / battery powered heater': 'fuel_battery_powered_heater',
    'Parking camera: 360-degree camera': 'parking_camera_360-degree_camera',
    'Battery preheating': 'battery_preheating',
    'Electric seats: Without memory': 'electric_seats_without_memory',
    'Air suspension': 'air_suspension',
    'Curve lights': 'curve_lights',
    'Head-Up display': 'head_up_display',
    'Sport base': 'sport_base,'
}

DRIVE_TYPE = {'Four wheel': '4wd', 'Front wheel': 'fwd', 'Rear wheel': 'rwd', 'Not available': None}

fields_to_pick = [
    'make',
    'model',
    'modelTypeName',
    'color',
    'driveType',
    'price',
    'totalOwners',
    'kilometers',
    'seats',
    'power',
    'batteryCapacity',
    'electricRange',
]

columns = [
    *fields_to_pick,
    'isSuv',
    'metallicColor',
    *ACCESSORIES_TO_COLUMN_NAME.values(),
]

In [2]:
import re

zero_times_s = 'Pro Performance 1ST 150 kW'
one_times_s = 'Pro Performance 1ST 150 kW, akku 77 kWh'
two_times_s = 'Pro Performance 1ST 150 kW, akku 77 kWh Pro Performance 1ST 150 kW, akku 88 kWh'
one_time_no_space = 'Pro Performance 1ST 150 kW, akku 77kWh Pro Performance 1ST 150 kW'



battery_regex = r'\d{2,3}\s*kWh'

def parse_battery_capacity_from_free_text(free_text):
    matches = re.findall(battery_regex, free_text, re.IGNORECASE)
    if not len(matches):
        return 0
    return sorted([int(s.lower().replace('kwh', '').split()[0]) for s in matches], reverse=True)[0]

print(parse_battery_capacity_from_free_text(zero_times_s))
print(parse_battery_capacity_from_free_text(one_times_s))
print(parse_battery_capacity_from_free_text(two_times_s))
print(parse_battery_capacity_from_free_text(one_time_no_space))

0
77
88
77


In [3]:
import os
import json
import pandas as pd

QUERY_RESULTS_PATH = './query_results'



def open_nettiauto_file(file_name):
    print(file_name)
    with open(f'{QUERY_RESULTS_PATH}/{file_name}', 'r') as f:
        data = json.load(f)
    return data

def map_nettiauto_entry_to_row(item):
    try:
        acc = item['accessories'] if item['accessories'] else []
        acc_set = set([ ACCESSORIES_TO_COLUMN_NAME[row['en']] for row in acc if row['en'] in ACCESSORIES_TO_COLUMN_NAME])


        result_without_accessories = {key: item[key] for key in fields_to_pick if key in item}
        result_without_accessories['isSuv'] = item['bodyType']['id'] == 5
        result_without_accessories['make'] = result_without_accessories['make']['name'].lower()
        result_without_accessories['model'] = result_without_accessories['model']['name'].lower()
        result_without_accessories['color'] = result_without_accessories['color']['en'].lower()
        result_without_accessories['metallicColor'] = item['colorType']['en'].lower() == 'metallic'
        result_without_accessories['driveType'] = DRIVE_TYPE[result_without_accessories['driveType']['en']]
        
        # It's ugly but does its job. I was able to get the missing batteryCapacity from 2109 to 1219
        if not result_without_accessories['batteryCapacity'] and 'modelTypeName' in item and item['modelTypeName']:
            result_without_accessories['batteryCapacity'] = parse_battery_capacity_from_free_text(item['modelTypeName'])
        if not result_without_accessories['batteryCapacity'] and 'description' in item and item['description']:
            result_without_accessories['batteryCapacity'] = parse_battery_capacity_from_free_text(item['description'])
        if not result_without_accessories['modelTypeName']:
            result_without_accessories['modelTypeName'] = ''

        return {
            **result_without_accessories,
            **{key: key in acc_set for key in ACCESSORIES_TO_COLUMN_NAME.values()},
        }
    except Exception as e:
        print(item['id'])
        raise e


dfs = []

for file_name in sorted(os.listdir(QUERY_RESULTS_PATH)):
    results = map(map_nettiauto_entry_to_row, open_nettiauto_file(file_name))
    dfs.append(pd.DataFrame(data=results, columns=columns))

    
df = pd.concat(dfs)
df.head()


2021-01_response_1696071482757.json
2021-02_response_1696071737045.json
2021-03_response_1696072016857.json
2021-04_response_1696072267024.json
2021-05_response_1696072588298.json
2021-06_response_1696072850927.json
2021-07_response_1696073239785.json
2021-08_response_1696073507847.json
2021-09_response_1696073655890.json
2022-01_response_1696074468991.json
2022-02_response_1696074615821.json
2022-03_response_1696074839641.json
2022-04_response_1696075021170.json
2022-05_response_1696075239764.json
2022-07_response_1696075876798.json
2022-07_response_1696079148457.json
2022-08_response_1696076147323.json
2022-09_response_1696076364847.json
2022-10_response_1696076596294.json
2022-11_response_1696076798891.json
2022-12_response_1696076991892.json
2022-13_response_1696077143377.json
2022-14_response_1696077274819.json
2022_06_response_1696075611367.json
2023-01_response_1696078129488.json
2023-02_response_1696078250632.json
2023-03_response_1696078413236.json
2023-04_response_16960785579

,make,model,modelTypeName,color,driveType,price,totalOwners,kilometers,seats,power,...,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,"sport_base,"
0,porsche,taycan,Taycan sähköauto Heti toimitukseen!! rahoitus ...,black,rwd,101880.0,1.0,2100.0,5.0,300.0,...,False,True,False,False,False,True,False,False,False,True
1,opel,zafira-e,"Life L Comfort 136 automaatti 50 **Täyssähkö, ...",other,fwd,47900.0,NaN,2500.0,NaN,100.0,...,False,False,False,False,False,False,False,False,False,False
2,mercedes-benz,eqs,"580 4Matic ** Muistipenkit, Burmester, nelipyö...",gray,4wd,125900.0,NaN,30000.0,5.0,385.0,...,True,True,True,True,True,False,True,False,False,False
3,nissan,leaf,e+ N-Connecta MY21 62 kWh *Isoakkunen Huippuva...,black,fwd,35800.0,NaN,10000.0,5.0,160.0,...,False,False,False,False,False,False,False,False,False,False
4,citroen,e-c4,"Full Electric 136 Shine 50 kWh, KORKOTARJOUS 3...",blue,fwd,33800.0,NaN,17000.0,5.0,100.0,...,False,False,False,False,False,False,False,False,False,False


In [34]:
def update_capacity(df, make, model, capacity, re=".*", reverse=False):
    m = ((df["make"] == make)
        & (df["model"] == model))
    if reverse:
        m = (m & ~((df["modelTypeName"].str.contains(re, case=False, regex=True) == True).values))
    else:
        m = (m & ((df["modelTypeName"].str.contains(re, case=False, regex=True) == True).values))

    df.loc[m,"batteryCapacity"] = capacity
    print(f"Updated {m.sum()} cars with parameters:\t{make} {model} {re}, {capacity}")
    

def get_missing_df(df, make, model=''):
        if model:
                return df[(df['batteryCapacity'] == 0) & (df['make'] == make) & (df['model'] == model)][['make', 'model', 'modelTypeName', 'power', 'kilometers', 'electricRange']]
        return df[(df['batteryCapacity'] == 0) & (df['make'] == make)][['make', 'model', 'modelTypeName', 'power', 'kilometers', 'electricRange']]

def print_info(df, make, model, re=".*"):
        mask = (df['make'] == make) & (df['model'] == model) & (df['modelTypeName'].str.contains(re, regex=True))
        mercedes = df[mask]
        print(mercedes['batteryCapacity'].value_counts())
        print(mercedes[['modelTypeName', 'batteryCapacity']])
        
def print_missing_for_make(df, make):
        no_battery_df = df[df['batteryCapacity'] == 0]
        mercedes_no_battery = no_battery_df[no_battery_df['make'] == make]['model']
        print(mercedes_no_battery.value_counts())
        
def print_make_missing_battery(df):
    print(df[df['batteryCapacity'] == 0]['make'].value_count())

In [5]:
update_capacity(df, "toyota", "bz4x", 64) 

no_battery_df = df[df['batteryCapacity'] == 0]
toyotas_no_battery = no_battery_df[no_battery_df['make'] == 'toyota']['model']
toyotas_no_battery.value_counts()
get_missing_df(df, 'toyota')
print_missing_for_make(df, 'toyota')

Updated 51 cars with parameters:	toyota bz4x .*, 64
Series([], Name: count, dtype: int64)


In [6]:
mercedes = df[(df['make'] == 'mercedes-benz') & (df['model'] == 'eqe')]
print(mercedes['batteryCapacity'].value_counts())
mercedes['modelTypeName']

batteryCapacity
0.0      78
90.0     25
89.0     24
90.6     11
100.0     7
91.0      3
98.0      2
88.1      1
56.0      1
Name: count, dtype: int64


16    350+ #Suomi-auto  #Panoraamalasikatto #Premium...
17    Mercedes-AMG EQE 43 4MATIC #Suomi-auto #Premiu...
18    350+ #AMG Line #Suomi-auto #Premium #Digital L...
19                      350+ *** Korko alk. 1,95% !!! *
43    350+ #Suomi-auto #Premium #AMG-ulko-/sisä #AMG...
                            ...                        
89    500 4MATIC SUV ** Esittelyauto nopeaan toimitu...
92                                           350 4MATIC
97    350 4matic ** Esittelyauto nopeaan toimituksee...
7                                        350 4MATIC SUV
14    500 4MATIC SUV #Heti toimitus #AMG-Line #Nelip...
Name: modelTypeName, Length: 152, dtype: object

In [7]:
update_capacity(df, "mercedes-benz", "eqe", 89.0, r'(350|300)\s') 
update_capacity(df, "mercedes-benz", "eqe", 90.6, r'(350\+|500|43)\s') 
get_missing_df(df, "mercedes-benz", 'eqe')


Updated 67 cars with parameters:	mercedes-benz eqe (350|300)\s, 89.0
Updated 64 cars with parameters:	mercedes-benz eqe (350\+|500|43)\s, 90.6


/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_65335/1346181111.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = (m & ((df["modelTypeName"].str.contains(re, case=False, regex=True) == True).values))


,make,model,modelTypeName,power,kilometers,electricRange
49,mercedes-benz,eqe,"2xAMG-Line, Distronic+, Panorama, Muistipakett...",215.0,7500.0,NaN
99,mercedes-benz,eqe,,215.0,9300.0,NaN
5,mercedes-benz,eqe,,215.0,NaN,NaN
51,mercedes-benz,eqe,"AMG-Line SE-edition, hinta sis.alv 24%, vetoko...",180.0,6000.0,NaN
51,mercedes-benz,eqe,,215.0,NaN,NaN
43,mercedes-benz,eqe,,215.0,9000.0,NaN
52,mercedes-benz,eqe,"Advanced Sound System, Nahkaverhoilu, LED High...",180.0,10500.0,NaN
23,mercedes-benz,eqe,,215.0,NaN,NaN
69,mercedes-benz,eqe,,215.0,NaN,NaN
11,mercedes-benz,eqe,,215.0,NaN,NaN


In [8]:
update_capacity(df, "mercedes-benz", "eqs", 108.4, r'(450|450\+|580|53)\s') 
update_capacity(df, "mercedes-benz", "eqs", 90.6, r'(350)\s') 
get_missing_df(df, "mercedes-benz", 'eqs')

Updated 69 cars with parameters:	mercedes-benz eqs (450|450\+|580|53)\s, 108.4
Updated 8 cars with parameters:	mercedes-benz eqs (350)\s, 90.6


/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_65335/1346181111.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = (m & ((df["modelTypeName"].str.contains(re, case=False, regex=True) == True).values))


,make,model,modelTypeName,power,kilometers,electricRange
29,mercedes-benz,eqs,"Huippu Upea uutuus! Neliveto! Panorama, 360°, ...",385.0,14000.0,NaN
52,mercedes-benz,eqs,,385.0,10000.0,NaN
32,mercedes-benz,eqs,,385.0,NaN,NaN
58,mercedes-benz,eqs,,265.0,1.0,NaN
34,mercedes-benz,eqs,,265.0,NaN,NaN
18,mercedes-benz,eqs,,400.0,NaN,NaN
57,mercedes-benz,eqs,,400.0,10000.0,NaN
73,mercedes-benz,eqs,,265.0,NaN,NaN
58,mercedes-benz,eqs,,NaN,NaN,NaN
28,mercedes-benz,eqs,,400.0,NaN,NaN


In [9]:
# https://www.mercedes-benz.fi/content/dam/finland/passengercars/NEW-WLTP-pricelists/EQA/EQA_10.5.2023_1.pdf
print_info(df, 'mercedes-benz', 'eqa', r'\d{3}\+')
update_capacity(df, "mercedes-benz", "eqa", 70.5, r'\d{3}\+') 
update_capacity(df, "mercedes-benz", "eqa", 66.5, r'\d{3}') 
get_missing_df(df, "mercedes-benz", 'eqa')

batteryCapacity
66.0     1
100.0    1
70.0     1
0.0      1
Name: count, dtype: int64
63    250+ suuremmalla akulla Advanced 529km (WLTP) ...
15    250+ Business / Koukku / AMG Line / EQ-Navigoi...
26     250+ Business **Distronic, MBUX-inovaatiopaketti
17                                        250+ Business
Name: modelTypeName, dtype: object
Updated 4 cars with parameters:	mercedes-benz eqa \d{3}\+, 70.5
Updated 73 cars with parameters:	mercedes-benz eqa \d{3}, 66.5


,make,model,modelTypeName,power,kilometers,electricRange
72,mercedes-benz,eqa,* Amg paketoitu ( ! ) *,NaN,29000.0,NaN
98,mercedes-benz,eqa,,NaN,NaN,NaN
49,mercedes-benz,eqa,,140.0,23000.0,420.0
56,mercedes-benz,eqa,,168.0,7000.0,NaN
72,mercedes-benz,eqa,,NaN,3000.0,NaN
4,mercedes-benz,eqa,,5.0,NaN,NaN


In [10]:
# https://www.mercedes-benz.fi/content/dam/finland/passengercars/NEW-WLTP-pricelists/EQA/EQA_10.5.2023_1.pdf
print_info(df, 'mercedes-benz', 'eqb', r'\d{3}\+')
update_capacity(df, "mercedes-benz", "eqb", 70.5, r'\d{3}\+') 
update_capacity(df, "mercedes-benz", "eqb", 66.5, r'\d{3}') 
get_missing_df(df, "mercedes-benz", 'eqb')

Series([], Name: count, dtype: int64)
Series([], Name: modelTypeName, dtype: object)
Updated 0 cars with parameters:	mercedes-benz eqb \d{3}\+, 70.5
Updated 78 cars with parameters:	mercedes-benz eqb \d{3}, 66.5


,make,model,modelTypeName,power,kilometers,electricRange
50,mercedes-benz,eqb,,NaN,NaN,NaN
62,mercedes-benz,eqb,,168.0,8090.0,NaN
75,mercedes-benz,eqb,,140.0,NaN,NaN
99,mercedes-benz,eqb,,5.0,6000.0,NaN
22,mercedes-benz,eqb,,NaN,NaN,NaN
41,mercedes-benz,eqb,,168.0,3000.0,NaN
85,mercedes-benz,eqb,,5.0,NaN,NaN


In [11]:
print_info(df, 'mercedes-benz', 'eqc')
update_capacity(df, "mercedes-benz", "eqc", 80) 
get_missing_df(df, "mercedes-benz", 'eqc')

batteryCapacity
80.0    25
0.0     23
82.0     1
Name: count, dtype: int64
15                                           400 4Matic
41    400 4Matic AMG Line, Head up !! Distronic !! 3...
74                   takuu 11/23, Rahoitus alkaen 3,99%
16           400 AMG 4Matic // ALV / Huippuvarusteet //
34    400 4Matic AMG / Nahkaverhoilu / Keyless Go / ...
95                                                     
0     AMG 400 4Matic / Designo sisusta ja ulkoväri!!...
34    400 4Matic * AMG Edition, Distronic+, 360 kame...
7     400 4MATIC AMG / DTR+ / Koukku / WideScreen / ...
11    400 4Matic AMG-Line*Night Desing*Burmester / D...
26    400 4MATIC, WLTP 472km, Tunnelmavalaistus, 360...
28    400 4Matic AMG **Distonic plus / Multibeam LED...
33    400 4Matic Edition AMG-Styling! Akkutesti teht...
61    400 4Matic Edition AMG - Facelift-malli, Huipp...
80                                                     
13                                                     
19    400 4MATIC Business AMG

,make,model,modelTypeName,power,kilometers,electricRange


In [12]:
print_missing_for_make(df, 'mercedes-benz')

model
eqe        12
eqs        10
eqs suv     8
eqb         7
eqa         6
eqv         3
vito        1
Name: count, dtype: int64


In [13]:
print_info(df, 'bmw', 'ix')
update_capacity(df, "bmw", "ix", 76.6, r'xDrive\s*40')
update_capacity(df, "bmw", 'ix', 111.5, r'xDrive\s*40', True)
get_missing_df(df, "bmw", 'ix')


batteryCapacity
0.0      33
71.0      5
111.0     4
76.6      3
77.0      3
76.0      3
111.5     2
105.0     1
Name: count, dtype: int64
92    xDrive40 // BMW Premium Selection -takuu 24kk/...
20    xDrive40 Fully Charged / Adapt.cruise / HUD / ...
28    xDrive40 *SUPERVARUSTEET! 22"/LASER/HUD/PANORA...
88    xDrive40 Fully Charged, WLTP 372km, Harman/Kar...
55    xDrive50 //H&K/Laser/Ilmajousitus/Nelipyöräohj...
71    xDrive40 Fully Charged 11CF *** Korko alk. 1,9...
88    xDrive40, WLTP 372km, Sähkösäätöiset Etuistuim...
15    xDrive40 Fully Charged *ESITTELYAUTO* Hinta uu...
42    xDrive40 Fully Charged // Huippuvarusteet  **B...
44     xDrive50 Signature *** Tämä auto nyt etuhintaan!
62          xDrive40 *** BMW Rahoitusetu 2,90% (+kulut)
63    Akku 77kWh xDrive40 Fully Charged Aut. 326hv |...
22    1-om, M-Sport, HUD, Panorama, Harman&Kardon Ra...
85    xDrive40, WLTP 372km, HUD, Kamera, Sähkösäätöi...
12    xDrive40 Fully Charged // Sport paketti/Ajoavu...
82    xDrive40 Fully C

,make,model,modelTypeName,power,kilometers,electricRange


In [14]:
print_info(df, 'bmw', 'ix')
update_capacity(df, "bmw", "ix", 76.6, r'xDrive\s*40')
update_capacity(df, "bmw", 'ix', 111.5, r'xDrive\s*40', True)
get_missing_df(df, "bmw", 'ix')

batteryCapacity
76.6     31
111.5    23
Name: count, dtype: int64
92    xDrive40 // BMW Premium Selection -takuu 24kk/...
20    xDrive40 Fully Charged / Adapt.cruise / HUD / ...
28    xDrive40 *SUPERVARUSTEET! 22"/LASER/HUD/PANORA...
88    xDrive40 Fully Charged, WLTP 372km, Harman/Kar...
55    xDrive50 //H&K/Laser/Ilmajousitus/Nelipyöräohj...
71    xDrive40 Fully Charged 11CF *** Korko alk. 1,9...
88    xDrive40, WLTP 372km, Sähkösäätöiset Etuistuim...
15    xDrive40 Fully Charged *ESITTELYAUTO* Hinta uu...
42    xDrive40 Fully Charged // Huippuvarusteet  **B...
44     xDrive50 Signature *** Tämä auto nyt etuhintaan!
62          xDrive40 *** BMW Rahoitusetu 2,90% (+kulut)
63    Akku 77kWh xDrive40 Fully Charged Aut. 326hv |...
22    1-om, M-Sport, HUD, Panorama, Harman&Kardon Ra...
85    xDrive40, WLTP 372km, HUD, Kamera, Sähkösäätöi...
12    xDrive40 Fully Charged // Sport paketti/Ajoavu...
82    xDrive40 Fully Charged, WLTP 372km, Innovation...
83    xDrive40 Fully Charged // H&K/ S

,make,model,modelTypeName,power,kilometers,electricRange


In [15]:
print_info(df, 'bmw', 'ix1')
update_capacity(df, "bmw", "ix1", 66.5)
get_missing_df(df, "bmw", 'ix1')

batteryCapacity
0.0     33
68.0    13
64.0     2
64.7     1
64.8     1
66.0     1
67.0     1
Name: count, dtype: int64
2              xDrive30 61EF *** Korko alk. 1,95% !!! *
5              xDrive30 61EF *** Korko alk. 1,95% !!! *
66    U11 30 xDrive Charged Edition - *Heti toimituk...
73                                M Sport U11 30 xDrive
50    U11 30 xDrive M-Sport - Adaptiviinen vakionope...
43    30 xDrive, WLTP 440km, M-Sport, Kamera, Drivin...
74    30e xDrive /Driving assistant plus /Harman Kar...
87             xDrive30 61EF *** Korko alk. 1,95% !!! *
81             xDrive30 61EF *** Korko alk. 1,95% !!! *
63    U11 30 xDrive (M Sport) *** BMW Rahoitusetu 2,...
74    U11 30 xDrive Charged Edition *** Tämä auto ny...
2          U11 30 xDrive xLine, täyssähkö esittelyauto!
24             xDrive30 61EF *** Korko alk. 1,95% !!! *
66    U11 30 xDrive *ESITTELYAUTO* Hinta uutena ~69 ...
83             xDrive30 61EF *** Korko alk. 1,95% !!! *
28    xDrive30 xLine Charged Plus *** Kor

,make,model,modelTypeName,power,kilometers,electricRange


In [16]:
print_info(df, 'bmw', 'i4')
update_capacity(df, "bmw", "i4", 83.9)
get_missing_df(df, "bmw", 'i4')

batteryCapacity
83.0    32
0.0     29
81.0     5
83.9     5
84.0     4
80.0     2
80.7     1
Name: count, dtype: int64
7                     eDrive40 M-Sport *HUD*ACC*Kamera*
18    eDrive 40 M-Sport, WLTP 590km, Adaptiivinen Va...
75    eDrive40 M Sport Pro #Jatkotakuu #ACC #Harman/...
8     M-Sport / Aktiivinen vakionopeudensäädin / Per...
46    eDrive40 M-Sport * Takuu 02/2024 / Harman Kard...
                            ...                        
58    M50 Super Charged *** BMW Rahoitusetu 2,90% (+...
77                                                  M50
78          eDrive40 *** BMW Rahoitusetu 2,90% (+kulut)
80    M50 // Hinta uutena ~ 78 510€ // Luovutus 12 /...
90                                     eDrive40 Charged
Name: modelTypeName, Length: 78, dtype: object
Updated 78 cars with parameters:	bmw i4 .*, 83.9


,make,model,modelTypeName,power,kilometers,electricRange


In [17]:
print_info(df, 'bmw', 'i4 m50')
update_capacity(df, "bmw", "i4 m50", 83.9)
get_missing_df(df, "bmw", 'i4 m50')

batteryCapacity
0.0     25
83.0    13
83.9     5
84.0     4
81.0     4
82.0     1
Name: count, dtype: int64
9     M50 // Driving assistant+/ Laser/ Comfort acce...
77    xDrive - M-Sport, Adaptiivinen alusta - sähkös...
80    Laserajovalot, Driving assistant+, Hifi, HUD, ...
87    G26 M50 * Korko 4,99% / ACC / Harman Kardon / ...
41                                               M50 i4
62    i4 M50 - Korkokampanja 3.99%+kulut - Suomiauto...
99    M50 - *Korko 3,99%+kulut + Kotiintoimitus alka...
80    X-Drive - M-Sport -Carbon -pack - 20” Multispo...
83    ** Vetokoukku / HIFI / BMW Live Cockpit Plus /...
14                                 M50 i4 Fully Charged
76    Huippu varusteet! BMW Individual metalliväri, ...
35    M50 // Takuu / ACC / Kamera / M Sport - pakett...
44    Super Charged - Frozen Portimao Blue, Supervar...
83    xDrive - M Brooklyn Grey metallic / M Sport Ae...
32                                 M50 i4 Fully Charged
33    M50//M sport paketti / M Sport alusta / Hifit 

,make,model,modelTypeName,power,kilometers,electricRange


In [18]:
update_capacity(df, "bmw", "ix3", 80.0)
update_capacity(df, "bmw", "i7", 101.7)
update_capacity(df, "bmw", "ix m50", 111.5)

print_missing_for_make(df, 'bmw')

Updated 39 cars with parameters:	bmw ix3 .*, 80.0
Updated 9 cars with parameters:	bmw i7 .*, 101.7
Updated 0 cars with parameters:	bmw ix m50 .*, 111.5
model
i3        4
i3s       3
ix m60    3
Name: count, dtype: int64


In [19]:
update_capacity(df, "tesla", "model 3", (54+55)/2, "standard[-|\s]*range") 
update_capacity(df, "tesla", "model 3", ((78+79.5)/2), "long[\s|-]range|performance")
get_missing_df(df, "tesla", "model 3")


Updated 37 cars with parameters:	tesla model 3 standard[-|\s]*range, 54.5
Updated 90 cars with parameters:	tesla model 3 long[\s|-]range|performance, 78.75


,make,model,modelTypeName,power,kilometers,electricRange
23,tesla,model 3,",Refresh, Eap, ilp, Chrome delete, 20""Uber tur...",377.0,25000.0,NaN
25,tesla,model 3,*Tummennukset*Ilp*,324.0,48000.0,NaN
37,tesla,model 3,,340.0,20000.0,NaN
68,tesla,model 3,"ILP, Sähk. takaluukku, Panoramakatto, Autopilo...",211.0,35000.0,NaN
35,tesla,model 3,*Vaihto *Rahoitus *Takuu,340.0,28991.0,NaN
87,tesla,model 3,"Facelift, Tehdastakuu, Autopilot, KW alusta, K...",513.0,36000.0,NaN
29,tesla,model 3,,239.0,28000.0,NaN
39,tesla,model 3,SR+ / Autopilot / Premium Audio / ILP / Panor...,NaN,123000.0,NaN
55,tesla,model 3,,NaN,21340.0,NaN
56,tesla,model 3,,510.0,45285.0,NaN


In [20]:
update_capacity(df, "tesla", "model y", 78.1, "standard[-|\s]*range")
update_capacity(df, "tesla", "model y", 78.1, "long[\s|-]range|performance")
get_missing_df(df, "tesla", "model y")

Updated 0 cars with parameters:	tesla model y standard[-|\s]*range, 78.1
Updated 111 cars with parameters:	tesla model y long[\s|-]range|performance, 78.1


,make,model,modelTypeName,power,kilometers,electricRange
66,tesla,model y,,NaN,58000.0,NaN
29,tesla,model y,Model Y Monikäyttöajoneuvo (AF) 5ov - MODEL Y ...,5.0,22000.0,NaN
99,tesla,model y,Prem. Audio & Nahat / ilmalämpöpumppu/autopilotti,258.0,32100.0,NaN
0,tesla,model y,"* Valkoinen * 20"" Induction vanteet * AMD Ryze...",258.0,9300.0,NaN
8,tesla,model y,* Vetokoukku * AMD ryzen * ilmalämpöpumppu *,340.0,18000.0,NaN
75,tesla,model y,Rear-Wheel Drive RWD Aut.| Vetokoukku | AMD Ry...,NaN,26400.0,NaN
92,tesla,model y,"Parannettu Autopilot, Teslan irrotettava Vetok...",514.0,49000.0,NaN
48,tesla,model y,,534.0,11000.0,NaN
62,tesla,model y,VAIHTO / RAHOITUS!!,336.0,26000.0,520.0
70,tesla,model y,"Vetokoukku, Midnight Cherry Red",336.0,5900.0,NaN


In [24]:
print_missing_for_make(df, 'tesla')

model
model 3    19
model y    13
model s     3
model x     1
Name: count, dtype: int64


In [27]:
# https://www.media.volvocars.com/global/en-gb/models/xc40-recharge/2023/specifications
# https://www.media.volvocars.com/global/en-gb/models/c40-recharge/2023/specifications
update_capacity(df, "volvo", "xc40", 78, "recharge\s*twin")
update_capacity(df, "volvo", "xc40", 69, "recharge")
update_capacity(df, "volvo", "c40", 78, "recharge\s*twin")
update_capacity(df, "volvo", "c40", 69, "recharge")

print_missing_for_make(df, 'volvo')

Updated 28 cars with parameters:	volvo xc40 recharge\s*twin, 78
Updated 68 cars with parameters:	volvo xc40 recharge, 69
Updated 57 cars with parameters:	volvo c40 recharge\s*twin, 78
Updated 77 cars with parameters:	volvo c40 recharge, 69
model
xc40    7
c40     4
Name: count, dtype: int64


In [29]:
# https://ev-database.org/car/1586/Opel-Mokka-e 50
# https://ev-database.org/uk/car/1585/Vauxhall-Corsa-e 50
update_capacity(df, "opel", "mokka-e", 50)
update_capacity(df, "opel", "corsa-e", 50)

print_missing_for_make(df, 'opel')

Updated 77 cars with parameters:	opel mokka-e .*, 50
Updated 28 cars with parameters:	opel corsa-e .*, 50
model
mokka       4
zafira-e    1
Name: count, dtype: int64


In [44]:
# https://en.wikipedia.org/wiki/Audi_Q4_e-tron
# https://ev-database.org/car/1355/Audi-e-tron-55-quattro 95
# https://en.wikipedia.org/wiki/Audi_Q8_e-tron
# https://ev-database.org/car/1420/Audi-e-tron-GT-quattro 93.4
# https://ev-database.org/car/1153/Audi-e-tron-GT-RS 93.4
update_capacity(df, "audi", "q4 e-tron", 55, "35")
update_capacity(df, "audi", "q4 e-tron", 82, "40|45|50")
update_capacity(df, "audi", "e-tron", 95, "55.*quattro")
update_capacity(df, "audi", "e-tron", 71, "50.*(q|g)uattro")
update_capacity(df, "audi", "e-tron", 95, "Sportback 55")
update_capacity(df, "audi", "e-tron", 95, "Sportback S")
update_capacity(df, "audi", "q8 e-tron", 95, "55.*quattro")
update_capacity(df, "audi", "q8 e-tron", 71, "50.*quattro")
update_capacity(df, "audi", "q8 e-tron", 95, "Sportback 55")
update_capacity(df, "audi", "q8 e-tron", 71, "Sportback 50")
update_capacity(df, "audi", "e-tron gt", 93.4)
update_capacity(df, "audi", "e-tron gt rs", 93.4)

print_missing_for_make(df, 'audi')

Updated 18 cars with parameters:	audi q4 e-tron 35, 55
Updated 98 cars with parameters:	audi q4 e-tron 40|45|50, 82
Updated 28 cars with parameters:	audi e-tron 55.*quattro, 95
Updated 15 cars with parameters:	audi e-tron 50.*(q|g)uattro, 71
Updated 4 cars with parameters:	audi e-tron Sportback 55, 95
Updated 10 cars with parameters:	audi e-tron Sportback S, 95
Updated 13 cars with parameters:	audi q8 e-tron 55.*quattro, 95
Updated 20 cars with parameters:	audi q8 e-tron 50.*quattro, 71
Updated 9 cars with parameters:	audi q8 e-tron Sportback 55, 95
Updated 13 cars with parameters:	audi q8 e-tron Sportback 50, 71
Updated 26 cars with parameters:	audi e-tron gt .*, 93.4
Updated 14 cars with parameters:	audi e-tron gt rs .*, 93.4
model
e-tron       2
q4 e-tron    2
q8 e-tron    1
Name: count, dtype: int64


/var/folders/3r/zp_fcf693r3g5yq1xc11l1j00000gn/T/ipykernel_65335/4278549353.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = (m & ((df["modelTypeName"].str.contains(re, case=False, regex=True) == True).values))


In [47]:
print_info(df, 'audi', 'q8 e-tron')


batteryCapacity
71.0     25
95.0     13
106.0     1
0.0       1
Name: count, dtype: int64
                                        modelTypeName  batteryCapacity
8                  Sportback 50 quattro Progress Plus             71.0
42  55 Sportback, WLTP 540km, Musta Optiikka -pake...            106.0
98                   50 quattro S line Launch Edition             71.0
6                 Sportback 50 quattro Launch Edition             71.0
12  Sportback e-tron Launch Edition S line 50 quat...             71.0
45                        Sportback 50 Launch Edition             71.0
46                          50 quattro Launch Edition             71.0
99  Sportback 50 S line Launch Edition *Head-Up Di...             71.0
3                    50 quattro s-line Launch Edition             71.0
9   Sportback 50 quattro 250kw Progress Plus Katta...             71.0
12  Sportback 50 quattro 250kw Progress Plus Todel...             71.0
20                 Sportback 50 S line Launch Edition     

In [48]:
# Kiat
# https://ev-database.org/car/1666/Kia-Niro-EV 68
# https://www.kia.com/content/dam/kwcms/kme/uk/en/assets/vehicles/EV6/Specification/Kia%20EV6%20Spec%20Sheet%20May%202021.pdf (ev6) 77.5
# https://ev-database.org/car/1288/Kia-e-Soul-64-kWh 67.5
# https://ev-database.org/car/1666/Kia-Niro-EV 68
# https://ev-database.org/car/1835/Kia-EV9-998-kWh-AWD 99.8
update_capacity(df, "kia", "niro", 68)
update_capacity(df, "kia", "ev6", 77.5)
update_capacity(df, "kia", "e-soul",  67.5)
update_capacity(df, "kia", "niro ev",  68)
update_capacity(df, "kia", "ev9", 99.8)

print_missing_for_make(df, 'kia')

Updated 108 cars with parameters:	kia niro .*, 68
Updated 136 cars with parameters:	kia ev6 .*, 77.5
Updated 39 cars with parameters:	kia e-soul .*, 67.5
Updated 4 cars with parameters:	kia niro ev .*, 68
Updated 4 cars with parameters:	kia ev9 .*, 99.8
Series([], Name: count, dtype: int64)


In [51]:
no_battery_df = df[df['batteryCapacity'] == 0]
no_battery_df['make'].value_counts()
print(sum(df['batteryCapacity'] > 0) / len(df))
print(sum(df['batteryCapacity'] == 0))

0.8972960151802657
433
